# Kaki King: World Renowned Artist and Virtuoso Guitarist

### “I don't remember a time that I didn't know how to play the guitar.”

<iframe width="560" height="315" src="https://www.youtube.com/embed/_7LX4FW7TEI?si=vEI_91rXJCq5OZYL" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

### With Rolling Stone Magazine's "Guitar God" Kaki King, we think about music and AI together.



## 🔥 Let's get the required packages

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.6.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.6.1-alpha"
#r "nuget: Microsoft.Extensions.Logging.Console, 8.0.0"
#r "nuget: NetCoreAudio, 2.0.0"
#r "nuget: SkiaSharp, 2.88.3"

In [ ]:
#!import ../config/Settings.cs
#!import ../config/Utils.cs
#!import ../config/SoundUtils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI) {
    kernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey)
        .Build();
} else {
    kernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion("gpt-4-1106-preview", apiKey, orgId)
        .Build();
}

## 🧑‍🍳 Let's get cooking together!

In [ ]:
string target = "an American country music style song";

## 🍲 Generating some sound ideas ...

In [ ]:
using System.IO;
using System.Net.Http;

string code = @"
This is a way to generate sounds in JS

const synth = new Tone.Synth().toDestination();
const now = Tone.now()
synth.triggerAttackRelease(""C4"", ""8n"", now)
synth.triggerAttackRelease(""E4"", ""8n"", now + 0.5)
synth.triggerAttackRelease(""G4"", ""8n"", now + 1)

The library is

https://cdnjs.cloudflare.com/ajax/libs/tone/14.8.49/Tone.js

Generate the music for ";

StringBuilder allUpdates = new StringBuilder();

await foreach (var update in kernel.InvokePromptStreamingAsync(code + target + " that gets played with a button on the web page. Think out loud how such a melody needs to sound."))
{
    Console.Write(update);
    allUpdates.Append(update); // Append each update to the StringBuilder
}

string allUpdatesString = allUpdates.ToString();

## 👀 Let's look at what the LLM said again

In [ ]:
string kk = Utils.WordWrap(allUpdatesString, 80);
Console.WriteLine(kk);

## 🌅 Let's make a relevant image

In [ ]:
#!import ../config/SkiaUtils.cs

#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0012


using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.TextToImage;

HttpClient httpClient = new HttpClient();

async Task DownloadImageAsync(string uri, string localPath)
{
    using (HttpResponseMessage response = await httpClient.GetAsync(uri))
    {
        response.EnsureSuccessStatusCode();

        using (Stream contentStream = await response.Content.ReadAsStreamAsync(), 
                fileStream = new FileStream(localPath, FileMode.Create, FileAccess.Write, FileShare.None))
        {
            await contentStream.CopyToAsync(fileStream);
        }
    }
}

Kernel painterKernel = Kernel.CreateBuilder()
    .AddOpenAITextToImage(apiKey) // Add your text to image service
    .Build();

ITextToImageService dallE = painterKernel.GetRequiredService<ITextToImageService>();

var image = await dallE.GenerateImageAsync(target, 256, 256);

Console.WriteLine(image);

string uniqueFileName = "abc-" + Guid.NewGuid().ToString() + ".png";
Console.WriteLine(uniqueFileName);
await DownloadImageAsync(image,uniqueFileName);
await SkiaUtils.ShowImage(image, 256, 256);

## 🕸️ Let's make a webpage to listen and watch it

In [ ]:
string code = @"Generate an HTML file from the following instructions. Do not include any explanatory text and just output the HTML file as is with no code block. Include all Javascript inline and include this image url in the file: " + image + " ";

StringBuilder allUpdates = new StringBuilder();

await foreach (var update in kernel.InvokePromptStreamingAsync(code + allUpdatesString))
{
    Console.Write(update);
    allUpdates.Append(update); // Append each update to the StringBuilder
}

string htmlFileString = allUpdates.ToString();

// write the string to a file called music.html
System.IO.File.WriteAllText("music.html", htmlFileString);
Console.WriteLine("music.html file written");

In [ ]:
http://127.0.0.1:5501/MusicAndAI/music.html